# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 14
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr14.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr14.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 211 211

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000222036,ENSG00000136319,ENSG00000129484,ENSG00000129566,ENSG00000100823,ENSG00000198805,ENSG00000214274,ENSG00000258818,ENSG00000169413,ENSG00000169397
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000222036,1.00000000,0.14542407,-0.1625138,-0.091769791,-0.068911018,0.12831280,-0.12615697,0.10013987,-0.0949200692,0.191652521
ENSG00000136319,0.14542407,1.00000000,-0.2690098,0.051849865,0.226900645,0.07946208,-0.03662723,-0.15875738,-0.0667898008,-0.007953435
ENSG00000129484,-0.16251380,-0.26900984,1.0000000,0.100498528,-0.024816099,0.18225727,0.09014197,-0.12657529,-0.0009649243,-0.036483335
ENSG00000129566,-0.09176979,0.05184987,0.1004985,1.000000000,-0.008927077,0.23415372,0.02979812,0.12951697,-0.0444864333,0.155347189
ENSG00000100823,-0.06891102,0.22690064,-0.0248161,-0.008927077,1.000000000,0.33566937,-0.05759616,0.10134789,0.0783831762,0.157656588
ENSG00000198805,0.12831280,0.07946208,0.1822573,0.234153716,0.335669371,1.00000000,-0.00811505,0.05748809,-0.0078236758,0.178713400


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 12

[1] -0.08315026 -0.11152108 -0.16782603 -0.20118147 -0.24159346 -0.33693757
 [7] -0.34438472 -0.35212088 -0.53475176 -0.60929844 -0.88710706 -1.21672454

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 211 211

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000222036,ENSG00000136319,ENSG00000129484,ENSG00000129566,ENSG00000100823,ENSG00000198805,ENSG00000214274,ENSG00000258818,ENSG00000169413,ENSG00000169397
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000222036,1.00000000,0.14542407,-0.1625138,-0.091769791,-0.068911018,0.12831280,-0.12615697,0.10013987,-0.0949200692,0.191652521
ENSG00000136319,0.14542407,1.00000000,-0.2690098,0.051849865,0.226900645,0.07946208,-0.03662723,-0.15875738,-0.0667898008,-0.007953435
ENSG00000129484,-0.16251380,-0.26900984,1.0000000,0.100498528,-0.024816099,0.18225727,0.09014197,-0.12657529,-0.0009649243,-0.036483335
ENSG00000129566,-0.09176979,0.05184987,0.1004985,1.000000000,-0.008927077,0.23415372,0.02979812,0.12951697,-0.0444864333,0.155347189
ENSG00000100823,-0.06891102,0.22690064,-0.0248161,-0.008927077,1.000000000,0.33566937,-0.05759616,0.10134789,0.0783831762,0.157656588
ENSG00000198805,0.12831280,0.07946208,0.1822573,0.234153716,0.335669371,1.00000000,-0.00811505,0.05748809,-0.0078236758,0.178713400


,ENSG00000222036,ENSG00000136319,ENSG00000129484,ENSG00000129566,ENSG00000100823,ENSG00000198805,ENSG00000214274,ENSG00000258818,ENSG00000169413,ENSG00000169397
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000222036,1.00000000,0.021553134,-0.06654744,0.010032050,0.01755387,0.02116274,-0.036498083,0.048625889,-0.008806632,0.05560755
ENSG00000136319,0.02155313,1.000000000,-0.12752309,-0.005907714,0.05483441,0.06687012,-0.066412881,-0.019038965,-0.024933904,0.02900879
ENSG00000129484,-0.06654744,-0.127523092,1.00000000,0.052926744,0.04305708,0.10363264,0.041287199,-0.016796254,0.037463939,-0.01473616
ENSG00000129566,0.01003205,-0.005907714,0.05292674,1.000000000,-0.03834056,0.10763357,0.016933688,0.088772125,-0.003286196,0.11362529
ENSG00000100823,0.01755387,0.054834413,0.04305708,-0.038340558,1.00000000,0.16930237,0.006604672,-0.003717205,0.009725710,0.03690845
ENSG00000198805,0.02116274,0.066870116,0.10363264,0.107633574,0.16930237,1.00000000,-0.042446873,0.051265066,0.012543079,0.06892646


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)